# Consigna

Link a la consigna: https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqrpsea4xf

A partir de la plataforma online (e-shop) de los países en los que opera, Nintendo tiene información de ventas de videojuegos diarias digitales por país en el siguiente RDD: (id_videojuego, codigo_pais, fecha, visitas_diarias, total_ventas_diarias). 

Por otro lado se tienen otro RDD que tiene información de todos los videojuegos que se venden en su plataforma con el siguiente formato (id_videojuego, titulo, rating_pegi, rating_esbr). Tener en cuenta que un mismo videojuego se puede vender en distintos países y esos nos permitirá obtener métricas a nivel global. 

Con esta información escribir un programa en pySpark que permita: 

a) Obtener el videojuego con más ventas digitales globales (es decir en todos los países) en un RDD con el siguiente formato: (id_videojuego, titulo, total), siendo total la cantidad total de ventas digitales globales

b) Para el videojuego con mas ventas, obtener cual es el país para el cual ser registra una mayor tasa de conversión (es decir, mayor total_ventas_diarias / visitas_diarias)

In [ ]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

In [ ]:
datos_ventas_diarias = [
    (1, 'AR', '2018-01-01', 30, 10),
    (1, 'ES', '2018-01-01', 23, 13),
    (2, 'US', '2018-01-04', 45, 5),
    (2, 'MX', '2018-01-04', 20, 10),
    (2, 'US', '2018-01-06', 50, 15),
    (3, 'AR', '2018-01-06', 10, 2),
    (1, 'US', '2018-01-06', 14, 4),
    (3, 'ES', '2018-01-10', 34, 11),
    (4, 'ES', '2018-01-11', 42, 24),
    (4, 'US', '2018-01-11', 83, 34),
    (4, 'AR', '2018-01-11', 27, 20),
    (4, 'MX', '2018-01-11', 47, 18),
    (4, 'AR', '2018-01-20', 10, 0),
    (4, 'US', '2018-01-21', 34, 2),
    (4, 'ES', '2018-01-21', 25, 7)
]

datos_videojuegos = [
    (1, 'Zelda: Breath of the Wild', 9, 8),
    (2, 'Mario Kart', 9, 7),
    (3, 'Splatoon 2', 11, 8),
    (4, 'Monster Hunter Generations Ultimate', 13, 10)
]

# Punto A

Sean:

* rdd_vd = rdd con los datos de ventas diarias de cada id_videojuego.
* rdd_vg = rdd con el título y rating asociado a cada id_videojuego.

In [ ]:
rdd_vd = sc.parallelize(datos_ventas_diarias)
rdd_vg = sc.parallelize(datos_videojuegos)

En este ejercicio necesitamos unir en algún momento ambos RDDs para poder juntar el id del videojuego más vendido a nivel mundial con su título correspondiente.

Esta unión se puede realizar al principio del ejercicio o al final, vamos a ver ambas versiones.

## Version realizando el join al principio.

### Paso 1

Para realizar el join no es necesario aplicar map a rdd_vg porque el join va a tomar el id_videojuego como clave, y titulo como valor, por ser las dos primeras columnas del RDD.

En caso de ser necesario, seria `rdd_vg.map(lambda x: (x[0], x[1]))`

Lo que sí debemos hacer es quedarnos con el id_videojuego y el total de ventas diaras de rdd_vd.

Luego del join nos quedarán registros`(id_videojuego, (total_ventas_diarias, titulo))`.

In [ ]:
rdd_unido = rdd_vd.map(lambda x: (x[0], x[4])).join(rdd_vg)
rdd_unido.collect()

### Paso 2

Cambiaremos el formato de las tuplas para poder hacer un reduceByKey y obtener el total de ventas anuales para cada video juego.

Nos quedarán con la pinta `((id_videojuego, titulo), total_ventas_diarias)`. De esta forma nuestras tuplas quedan con el formato correcto para poder realizar el reduceByKey.

Luego del reduceByKey haremos un map para pasarlas al formato pedido en la consigna.

In [ ]:
rdd_unido = rdd_unido.map(lambda x: ((x[0], x[1][1]), x[1][0]))
rdd_unido = rdd_unido.reduceByKey(lambda a,b: a+b).map(lambda x: (x[0][0], x[0][1], x[1]))
rdd_unido.collect()

### Paso 3

Para obtener la tupla con la mayor cantidad de ventas, simplemente buscamos el máximo utilizando reduce.

In [ ]:
juego_mas_vendido = rdd_unido.reduce(lambda a,b: a if a[2] > b[2] else b)
juego_mas_vendido

## Versión realizando el join al final.

En los criterios de corrección, se menciona que esta versión es la ideal.

In [ ]:
rdd_a = rdd_vd.map(lambda x: (x[0], x[4]))
rdd_a = rdd_a.reduceByKey(lambda a,b: a+b).reduce(lambda a,b: a if a[1] > b[1] else b)
rdd_a = sc.parallelize([rdd_a])
rdd_a = rdd_a.join(rdd_vg.map(lambda x: (x[0], x[1]))).map(lambda x: (x[0], x[1][1], x[1][0]))
rdd_a.collect()

# Punto B

### Paso 1

Para el rdd_vd quedo como valor tuplas de la pinta `(codigo_pais, total_vistas_diarias, total_ventas_diarias)`.

Hago un join con el resultado del punto A para quedarme con los registros del juego más vendido obtenido en el punto anterior.

Luego aplicamos un map para quedarnos con tuplas con la pinta `(codigo_pais, (total_vistas_diarias, total_ventas_diarias))`.

In [ ]:
rdd_b = rdd_vd.map(lambda x: (x[0], (x[1], x[3], x[4]))).join(rdd_a)
rdd_b = rdd_b.map(lambda x: (x[1][0][0], (x[1][0][1], x[1][0][2])))
rdd_b.collect()

### Paso 2

Ahora hacemos un reduceByKey para obtener los totales de ventas y vistas para cada pais.

Como valor tendremos tuplas `(total_vistas_diarias, total_ventas_diarias)`, por lo cual debemos sumar elemento a elemento.

In [ ]:
rdd_b = rdd_b.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))
rdd_b.collect()

### Paso 3

Utilizaremos map para calcular la tasa de conversión por país, quedando tuplas de la pinta `(codigo_pais, tasa)`.

In [ ]:
rdd_b = rdd_b.map(lambda x: (x[0], float(x[1][1]) / x[1][0]))
rdd_b.collect()

### Paso 4

Finalmente usamos reduce para obtener la máxima de las tasas de conversión.

In [ ]:
resultado_b = rdd_b.reduce(lambda a,b: a if a[1] > b[1] else b)
resultado_b